**计算插值算子的导数（一阶和二阶导数）**

In [ ]:
%reset -f

In [ ]:
import sys, os
sys.path.append(os.path.dirname(os.path.realpath('../utils')))
import numpy as np
from utils.aiwt import cjkfun, z_g, CompressInterp, adapt_CjJ4, restrain
from utils.iwavelets import dxphi0

In [ ]:
j0 = 0
J = 15
a = 0
b = 2
deltax = (b-a)/(2**J)
XJ0 = np.arange(a, b+deltax, deltax).reshape(1,-1)
V0 = np.sin(np.pi * XJ0)

eps0 = 0.005

M = 1
CC = 1

cjk, cjknum, zJc, zJcnum = cjkfun(V0, J, j0, a, b, eps0)
gjk, gjknum, gJc, gJcnum = z_g(j0, J, cjk, cjknum, zJc, zJcnum, a, b, M, CC)
XJ = a + deltax * gJc
V0 = CompressInterp(XJ0,V0,XJ)
C = adapt_CjJ4(J, j0, gjk, gjknum, gJc, gJcnum, a, b)

In [ ]:
def Ii1_2d(J, j0, cjk, cjknum, zJc, x, a, b, C):
    k = np.arange(2 ** j0+1).reshape((1,-1))
    rr = restrain(j0,k,J,zJc)
    ss1, ss2 = dxphi0(j0,k,a,b,x)
    Ii1d = np.dot(ss1, rr)
    Ii2d = np.dot(ss2, rr)

    for j in range(j0+1, J):
        if j == 0:
            continue
        elif j > j0 and cjknum[j] == 0:
            break
        k = cjk[j].reshape((1,-1))
        ss1, ss2 = dxphi0(j + 1,2 * k + 1,a,b,x)
        Ii1d = Ii1d + np.dot(ss1, C[j-1])
        Ii2d = Ii2d + np.dot(ss2, C[j-1])
    return Ii1d, Ii2d

In [ ]:
def theta1_2d(J, j0, a, b, C, cjk, cjknum, zJc, zJcnum):
    delta = (b - a) / (2 ** J)
    xk = a + zJc * delta
    xk = xk.reshape((1,-1))
    theta1d, theta2d = Ii1_2d(J, j0, cjk, cjknum, zJc, xk, a, b, C)
    return theta1d, theta2d

In [ ]:
theta1d, theta2d = theta1_2d(J, j0, a, b, C, gjk, gjknum, gJc, gJcnum)